In [ ]:
#----------------------------------------------------
# Code for find sources detected in the F335M Nircam image
# using Photutilts Find peaks: 
# https://photutils.readthedocs.io/en/latest/api/photutils.detection.find_peaks.html
#
#              Jimena Rodriguez
#                    2024
#---------------------------------------------------
# Import Functions
#---------------------------------------------------

import astropy
import photutils
import numpy as np
from astropy.io import fits, ascii
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy import wcs
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.stats import SigmaClip, sigma_clip
from photutils import Background2D, MedianBackground, SExtractorBackground, MMMBackground
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import SqrtStretch
from photutils import CircularAperture
from matplotlib.colors import LogNorm
from photutils.segmentation import detect_threshold
from photutils.detection import find_peaks
from photutils.centroids import centroid_2dg, centroid_1dg, centroid_com, centroid_quadratic
from astropy.stats import sigma_clipped_stats
import pandas as pd
from astropy.table import QTable

#----------------------------------------------------
#List of PHANGS-JWST cycle 1 galaxies
# and their distances from  
# Janice C. Lee et al 2023 ApJL 944 L17
#----------------------------------------------------


galaxy_name=['ngc5068', 'ic5332','ngc0628', 'ngc3351', 'ngc3627', 'ngc2835', 'ngc4254',
       'ngc4321', 'ngc4535', 'ngc1087', 'ngc4303', 'ngc1385', 'ngc1566',
       'ngc1433', 'ngc7496', 'ngc1512', 'ngc1300', 'ngc1672', 'ngc1365']

distances=[ 5.2 , 9.01, 9.84,  9.96, 11.32, 12.22, 13.1 , 15.21, 15.77, 15.85,16.99, 17.22, 17.69, 18.63, 18.72, 18.83, 18.99, 19.4 , 19.57]

#----------------------------------------------------

scale_v=[]
box=[]
filt=[]
# Directory where output files will be saved
outputdir='/Users/jrodriguez/Documents/JWST/JWST_Photometry/find_peaks/v1p1/' 
j=0

for galaxy in galaxy_name:
    

    DIR='/astro/phangshst/v1p1/'+galaxy.upper()+'/'  
    image,header=fits.getdata(DIR+galaxy+'_nircam_lv3_f335m_i2d_align.fits', header=True) # open image
    wcs=WCS(header)
#--------------------------------------------------------------------------
# Background estimation
# for more details read:
# https://photutils.readthedocs.io/en/latest/user_guide/background.html
#--------------------------------------------------------------------------
    scale=np.tan(np.pi/(3600*180))*distances[j]*1E6*0.063 #1px = scale pc (in NIRCAM 0.063"/px images)
    box_s_pc=85. # The background is estimated in boxes of (85pc)^2  
    box_s=int(np.rint(box_s_pc/scale)) # box size for background estimation in pixel
    filter_s_pc=15. # size of the median filter in pc
    filter_s=int(np.rint(filter_s_pc/scale)) # size of the median filter in pixels
    mod = filter_s % 2
    if mod==0:
        filter_s=filter_s+1
    
    scale_v.append(scale)
    box.append(box_s)
    filt.append(filter_s)
    mask = np.zeros(image.shape, dtype=bool)
    
    sigma_clip_bkg = SigmaClip(sigma=3., maxiters=15)
    bkg_estimator = SExtractorBackground()
    bkg = Background2D(image, (box_s,box_s), filter_size=(filter_s,filter_s),  
                       fill_value=0.0, edge_method='pad',
                       sigma_clip=sigma_clip_bkg, bkg_estimator=bkg_estimator, mask=mask)


#------------------------------------------------------------------------------
# Threshold 
#--------------------------------------------------------------------------

    sigma_clip_thr = SigmaClip(sigma=3., maxiters=5)
    threshold=detect_threshold(image, nsigma=3, background=bkg.background,
                     sigma_clip=sigma_clip_thr, mask=mask)

#----------------------------------------
# Source Detection
#----------------------------------------

    tbl = find_peaks(image, threshold,centroid_func=centroid_quadratic, box_size=3, wcs=wcs, mask=mask)
    tbl['peak_value'].info.format = '%.8g' 
    
    # Print the number of sources detected in each galaxy
    #----------------------------------------
    print('Number of sources in '+galaxy+'=',len(tbl))  
    
    # For sources where the centroid could not be determined, use the position of the peak instead.
    #----------------------------------------
    for i in range(len(tbl)):
        if np.isnan(tbl['x_centroid'][i]):
            tbl['x_centroid'][i]=tbl['x_peak'][i]
        
    for i in range(len(tbl)):
        if np.isnan(tbl['y_centroid'][i]):
            tbl['y_centroid'][i]=tbl['y_peak'][i]
            
    #convert from x,y in the image to  sky coordinates   
    #----------------------------------------
    sk = pixel_to_skycoord(tbl['x_centroid'], tbl['y_centroid'], wcs=wcs) 
    ra=sk.ra
    dec=sk.dec
    
    # built and save catalog 
    #----------------------------------------
    colnames=['x','y','raj2000','dej2000']
    coldata=[tbl['x_centroid'],tbl['y_centroid'],ra,dec]
    coord=QTable(coldata,names=colnames)
    coord.write(outputdir+galaxy+'_sources_f335m_findpeak.csv', serialize_method='data_mask', overwrite=True, format='ascii.csv')
    
    j+=1
